In [1]:
import os
import json
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console

In [2]:
console = Console()

In [3]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(OPENAI_API_KEY)

Success: .env file found with some environment variables

sk-proj-sBnfVCsPqLywAXCJBb-le46R5JzKpXTGvbqgFW7RfcfOHY8HQZ0GCF9iPHigrBa2KSg8uPy7LbT3BlbkFJ4MdGvmdgQQ9K8nfW7yfDQw8SO
UYOuj_4zJ-FYECe0m119fmBMAtB-snyiTK23PmdgmuL3P-RsA

In [4]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [5]:
conn = _conn_open()
cur = conn.cursor()

Conn: <connection object at 0x000001C3D056E790; dsn: 'user=postgres password=xxx dbname=postgres 
host=host.docker.internal', closed: 0>

In [6]:
FILENAME = "test_article_elements.json"
json_data = load_json(FILENAME)
print(f"Number of items: {len(json_data)}")
encoded_image_data = None
for i in range(len(json_data)):

    if json_data[i]["type"] == "Image":
        console.print(f"[dark_orange]---------Item Number IMAGE: {i + 1}--------[/]")
        el_image_b64 = json_data[i]["metadata"]["image_base64"]

        # console.print(el_image_b64[:40])
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_type, image_base64)
        VALUES 
        ('{new_id}','{new_element_id}','IMAGE','{el_image_b64}')
        RETURNING id;
        """
        # console.print(sql)
        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)
    if json_data[i]["type"] == "NarrativeText":

        console.print(f"[dark_orange]---------Item Number TEXT: {i + 1}--------[/]")
        el_text = json_data[i]["text"]
        el_embedding = get_embedding(el_text)

        # console.print(el_embedding)
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_text, element_type, image_base64, embedding)
        VALUES 
        ('{new_id}','{new_element_id}','{el_text}','NARRATIVE_TEXT',
        '{encoded_image_data}', '{el_embedding}')
        RETURNING id;
        """

        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)

Number of items: 12


---------Item Number IMAGE: 1--------

ID is cbdd9a21-adf6-4744-91d1-86aa0de10f9b

---------Item Number TEXT: 2--------

ID is e0e32c14-cdcb-4990-b207-ff94a727e223

...

ID is 0a9f7127-eaa5-4e6e-9e7f-e64036e3b2b6

---------Item Number TEXT: 11--------

ID is 77eb9536-7016-445f-90ba-dcb73f01eaab

---------Item Number TEXT: 12--------

ID is 7466e429-0e32-40cd-b152-5580c55fa04a

In [7]:
cur.close()
conn.close()